In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm
import quaternion
from comau_msgs.srv import SetIO, SetIORequest


# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=10, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
tool_change_pose = [-0.9376716018,  0.7174308896, -1.2232894897, -0.0096366024, -0.227122277 , -2.4522731304]

In [2]:
rospy.init_node('emergency_lamp_demo')

In [3]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
#pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()
    
    
def set_cutting_goal(pointA, pointB):
    material= MaterialType()
    material = material.PLASTIC

    normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
    normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")

    goal=CuttingGoal()
    goal.surface_normal_start=normalA
    goal.surface_normal_end=normalB
    goal.start_location=pointA
    goal.end_location=pointB
    goal.material.material_type=material
    goal.timeout=5000
    goal.thickness = 0.005 #not important. can be anything
    return goal
    
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)   
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)

### Load robot

In [4]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
#cobot.change_tool("nothing")

[rosout] [/emergency_lamp_demo] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/emergency_lamp_demo] connected to Matlab
Shutting down global node /matlab_global_node_39944 with NodeURI http://192.168.2.87:41577/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_39959 with NodeURI http://192.168.2.87:43533/
[rosout] [/emergency_lamp_demo] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [5]:
cobot.open_tool_changer()

In [ ]:
#!rosparam get /hrr_cobot/tool_name

cobot.change_tool("shaftgrinder")
cobot.tool_controller.run_shaftgrinder(1.0,"slow")

# Emergency Lamp disassembly

In [6]:
cobot.change_tool("nothing")

True

## Tool Change to Shaftgrinder

In [ ]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic = True)
# rospy.sleep(5)
# cobot.open_tool_changer()
# srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
# srv(SetIORequest(pin=14, state=False))
# srv(SetIORequest(pin=13, state=False))
# srv(SetIORequest(pin=12, state=False))
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
tool_change_client.send_goal(tc_goal)
#cancel(tool_change_client)

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cancel(tool_change_client)

## Cutting (6 goals)

### Clip back right

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.713196129333+0.0139-0.005, -0.185170262287+0.0220, 0.05-0.017], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.712934333216+0.0139-0.005, -0.175916732133+0.0220, 0.05-0.017], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)
# cobot.active_controllers

### Clip back middle

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.717783449157+0.0139-0.006, -0.0462650486226+0.0220+0.01, 0.05-0.017], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.717522325202+0.0139-0.006, -0.0170358101255+0.0220+0.01, 0.05-0.017], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)

### Clip back left

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.721999522931+0.003-0.005, 0.110633565404+0.03 , 0.05-0.017], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[ 0.722406576125+0.003-0.005, 0.119883059107+0.033, 0.05-0.017], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)

### Clip front left

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.55963279198-0.006, -0.189613837236+0.04,  0.05-0.017], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.560307534432-0.006, -0.179013482484+0.043,  0.05-0.017], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)

### Clip front middle

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[ 0.561264748141-0.004+0.003,-0.0444902111826+0.04,  0.05-0.017], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.562994113585-0.004+0.003, -0.0135911924867+0.043, 0.05-0.017], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)

### Clip front right

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.562622897175-0.008+0.003, 0.101301529051+0.04, 0.05-0.017], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.563292672634-0.008+0.003, 0.111233572438+0.043, 0.05-0.017], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)

In [ ]:
cobot.T_B_C_robot

## Move ROBOT

In [71]:
cobot.goTo(sm.SE3([0.0,0.0,0.1])@cobot.T_B_E_robot)

In [72]:
cobot.move_to_joint_pose([ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009], stochastic = True)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 82)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 82)
In HrRecyclerPerecptioPipeline (line 51)

calculating signed distance field ...
calculating signed distance field done
Elapsed time is 3.378141 seconds.
Elapsed time is 0.008868 seconds.


[rosout] [/emergency_lamp_demo] Planner is done and execution is running ...


In [ ]:
cobot.move_to_joint_pose(hier, stochastic = True)

### MAKE SURE TO UNFORCE!!

In [ ]:
cobot.change_tool("nothing")

In [7]:
cobot.tool

''

## Tool Change to Vacuum

In [ ]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic = True)
# cobot.change_tool("nothing")
# srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
# srv(SetIORequest(pin=14, state=False))
# srv(SetIORequest(pin=13, state=False))
# srv(SetIORequest(pin=12, state=False))
# rospy.sleep(5)
# cobot.open_tool_changer()

tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER
tool_change_client.send_goal(tc_goal)
# tool_change_client.cancel_all_goals()

## Vacuum Top

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
x: 0.651790252968
      y: -0.0348108583747
      z: -0.0348108583747

In [ ]:
obj_pos_guess = np.r_[0.651790252968 ,-0.0348108583747, 0.0348108583747] #[0.66, 0.17, 0.07] is actual vision data for elamp cover
obj_orient_guess = np.quaternion(-0.5, 0.5, 0.5, 0.5)
release_pos = np.r_[0.0192,0.5456 , 0.186] # will drop the object here, careful!
release_orientation = obj_orient_guess
#obj_orient_guess = quaternion.from_rotation_matrix(quaternion.as_rotation_matrix(obj_orient_guess) @ sm.SE3.Rx(np.rad2deg(-45)).R)

In [ ]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

def get_SE3_from_pose_stamped(msg):
    p = hrr_common.vec32np(msg.pose.position)
    q = hrr_common.quat2np(msg.pose.orientation)
    return hrr_common.posquat2homog(p, q)

object_center = get_SE3_from_pose_stamped(vac_goal.object_center)
release_center = get_SE3_from_pose_stamped(vac_goal.release_pose)
ee2tip = np.r_[0, 0, 0.20018]

_B_surface_normal = object_center.A[:3, 0]
grasp_pose_EE = hrr_common.utils.calc_goal_pose(_B_surface_normal, object_center.t, y_axis=object_center.A[:3, 1])
            #2. adjust to accomodate tool-tip
grasp_pose = grasp_pose_EE @ sm.SE3(-ee2tip)

release_pose_EE = hrr_common.utils.calc_goal_pose(release_center.A[:3,0], release_center.t, y_axis=release_center.A[:3,1])
release_pose = release_pose_EE @ sm.SE3(-ee2tip) 
#return self.cancel(msg="Thou shalt hard-code")

pre_pose = grasp_pose @ sm.SE3([0, 0, -0.15])

below_pose = grasp_pose @ sm.SE3([0, 0, 0.1])
pre_pose, below_pose

In [ ]:
vacuum_client.send_goal(vac_goal)


In [ ]:
cancel(vacuum_client)

In [67]:
cobot.close_tool_changer()


## Tool changing to adaptive gripper

In [8]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic = True)
# cobot.change_tool("nothing")
# rospy.sleep(3)
# cobot.open_tool_changer()

tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50_DSA
tool_change_client.send_goal(tc_goal)
# tool_change_client.cancel_all_goals()

In [ ]:
cobot.close_tool_changer()


In [13]:
cobot.change_tool("nothing")


True

In [14]:
cobot.change_tool("wsg_50_dsa")

True

## Calibrate

In [11]:
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = True
goal.scale_q4 = 0.5
goal.scale_q5 = 0.5
goal.scale_q6 = 0.5
goal

calibration_file: "/home/hrr_cobot/Documents/skill_recordings/wsg_50_dsa.pkl"
data_file: "/home/hrr_cobot/Documents/skill_recordings/wsg_50_dsa_calibration.npy"
q4_dot_des: 0.0
q5_dot_des: 0.0
q6_dot_des: 0.0
scale_q4: 0.5
scale_q5: 0.5
scale_q6: 0.5
recalibrate: True
keep_current_pose: False

In [12]:
calibrate_client.send_goal(goal)

In [ ]:
get_result(calibrate_client)

In [ ]:
# cancel(calibrate_client)

In [ ]:
cobot.FT.bias

In [ ]:
cobot.update()
cobot.FT.read_params()
cobot.update()

In [ ]:
cobot.B_F_msr

## Adaptive Grasping

In [ ]:
x: 0.641684953969
      y: -0.0272301237041
      z: 0.0484938847812
    orientation:
      x: 0.0
      y: -0.743050660019
      z: 0.0634292229665
      w: 0.666222523125

In [15]:
grasp_goal = AdaptiveGraspingGoal()
T_B_C_test = sm.SE3(0.641684953969, -0.0272301237041, 0.0484938847812)
dispose = sm.SE3([0.0192,0.5456 , 0.186])
grasp_goal.object_center.pose.position = hrr_common.np2vec3(T_B_C_test.t)
grasp_goal.object_center.pose.orientation = hrr_common.np2quat(hrr_common.rotmat2quat(T_B_C_test.R))
grasp_goal.dispose_off_location_pose.pose.position = hrr_common.np2vec3(dispose.t)
grasp_goal.dispose_off_location_pose.pose.orientation = hrr_common.np2quat(hrr_common.rotmat2quat(dispose.R))
grasp_goal.contact_force = 5.0
grasp_goal.timeout = 20000.0

In [16]:
grasping_client.send_goal(grasp_goal)

In [ ]:
cancel(grasping_client)

#Reset Gripper Driver
cobot.change_tool("nothing")
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.gripper.width

In [ ]:
#Test if gripper works
cobot.gripper.send_pos(0.05, si=True)

## Tool changing to vacuum

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic = True)

In [17]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic = True)
# rospy.sleep(2)
# cobot.open_tool_changer()
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER
tool_change_client.send_goal(tc_goal)
# tool_change_client.cancel_all_goals()

In [ ]:
cobot.close_tool_changer()
cobot.change_tool("vacuum")

## Vacuum Middle Cover

In [ ]:
x: 0.639662110735
      y: -0.0162336284559
      z: 0.0475931169759

In [53]:
obj_pos_guess = np.r_[0.639662110735 ,-0.0162336284559, 0.0475931169759] #[0.66, 0.17, 0.07] is actual vision data for elamp cover
obj_orient_guess = np.quaternion(-0.653281482438188, -0.270598050073099, 0.653281482438188, -0.270598050073099)
obj_orient_guess = quaternion.from_rotation_matrix(quaternion.as_rotation_matrix(obj_orient_guess) @ sm.SE3.Rx(np.rad2deg(45)).R)


release_pos = np.r_[0.0192,0.5456 , 0.186] # will drop the object here, careful!
release_orientation = obj_orient_guess

In [54]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

In [60]:
vacuum_client.send_goal(vac_goal)

In [57]:
cancel(vacuum_client)

In [58]:
cobot.change_tool("vacuum")

True

## Tool change to grinder

In [ ]:
cobot.move_to_joint_pose(tool_change_pose, stochastic = True)
rospy.sleep(2)
cobot.open_tool_changer()

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.close_tool_changer()

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.tool

## Grind battery cables

In [ ]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[ 0.508424204 , -0.0232085618,  0.0048460076], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[ 0.5339274748 , -0.0232085618,  0.0048460076], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
# cancel(shaftgrinder_client)

## Tool Change to Pneumatic Gripper

In [ ]:
cobot.move_to_joint_pose(tool_change_pose, stochastic=True)
cobot.change_tool("nothing")
cobot.open_tool_changer()

# tc_goal = ChangeToolGoal()
#tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
# tool_change_client.send_goal(tc_goal)
# tool_change_client.cancel_all_goals()

In [ ]:
srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
srv(SetIORequest(pin=14, state=False))
srv(SetIORequest(pin=13, state=False))
srv(SetIORequest(pin=12, state=False))

In [ ]:
cobot.close_tool_changer()

In [ ]:
cobot.change_tool("vacuum")

## Remove Battery

In [ ]:
obj_pos_guess = np.r_[ 0.594380102373, 0.211132105639+0.024, 0.0307231494051  ] #[0.712, 0.263, 0.0196] is actual vision data for battery
obj_orient_guess = np.quaternion(0.390420297656381, -0.197704307824811, 0.808372590594526, 0.393724208832667)#0.0,-0.4376,0.01394,0.8990) #actual vision data for battery
release_pos = np.r_[0.0192,0.5456 , 0.186] #same as vision
release_orientation = np.quaternion( 0.0026664697, -0.6740100594,  0.7380765346,  0.030762947 ) #same orientaiton for ease

In [ ]:
pgrip_goal = FingerGraspGoal()
pgrip_goal.object_center.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
pgrip_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

pgrip_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
pgrip_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

pgrip_goal.timeout = 10000.0
#pgrip_goal.gripper_open_width = 0.084
#pgrip_goal.gripper_close_width = 0

In [ ]:
pgrip_client.send_goal(pgrip_goal)

In [ ]:
pgrip_client.cancel_all_goals()

In [ ]:
cobot.change_tool("nothing")

## Tool Change to None

In [ ]:
cobot.stochastic_move_to_pose(tool_change_pose)
rospy.sleep(5)
cobot.open_tool_changer()
rospy.sleep(3)
cobot.close_tool_changer()
cobot.change_tool("nothing")
# tc_goal = ChangeToolGoal()
#tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
# tool_change_client.send_goal(tc_goal)
# tool_change_client.cancel_all_goals()